In [1]:
from itertools import combinations, permutations
import pandas as pd
from helpers import integer_linear_solver

2024-05-14 00:41:31.499 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
df = pd.read_csv("data/heroes.csv", index_col=0)


In [3]:
tokens = df.columns[1:-1].to_list()
tokens

['Teleporting',
 'Nuker',
 'Floating',
 'Healer',
 'Walking',
 'Mounted',
 'Crawling',
 'Running',
 'Ranged',
 'Flying',
 'Disabler',
 'Durable',
 'Initiator',
 'Pusher',
 'Jumping',
 'Slithering',
 'Escape',
 'Melee']

In [4]:

def trade4for1(tokens):
    # trade 4 tokens for 1, all combinations
    for trade_for in tokens:
        trade_tokens = tokens.copy()
        trade_tokens.remove(trade_for)
        combs = list(combinations(trade_tokens, 4))
        # print(f"Trade 4 tokens for {trade_for} ({len(combs)} combinations)")
        # row will be +1 for trade_for, -1 for trade_tokens, 0 for others
        rows = []
        for trade in combinations(trade_tokens, 4):
            row = [0] * len(tokens)
            row[tokens.index(trade_for)] = +1
            name_str = ""
            for t in trade:
                row[tokens.index(t)] = -1
                name_str += f"{t} "
            # last row has trading name
            rows.append([f"{name_str}for 1 {trade_for}"] + row)
    return rows

def trade3ofsamefor1(tokens):
    # trade 3 of same token for 1, all permutations
    rows = []
    for trade_for in tokens:
        trade_tokens = tokens.copy()
        trade_tokens.remove(trade_for)
        for trade in trade_tokens:
            row = [0] * len(tokens)
            row[tokens.index(trade_for)] = +1
            row[tokens.index(trade)] = -3
            # trade name
            rows.append([f"3 {trade} for 1 {trade_for}"] + row)
    return rows

trading_df = pd.DataFrame(trade3ofsamefor1(tokens), columns=["Trade"] + tokens) # trade3ofsamefor1(tokens)
# pd.DataFrame(trade4for1(tokens), columns=tokens)
# pd.DataFrame(trade3ofsamefor1(tokens), columns=tokens)
trading_df

,Trade,Teleporting,Nuker,Floating,Healer,Walking,Mounted,Crawling,Running,Ranged,Flying,Disabler,Durable,Initiator,Pusher,Jumping,Slithering,Escape,Melee
0,3 Nuker for 1 Teleporting,1,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3 Floating for 1 Teleporting,1,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3 Healer for 1 Teleporting,1,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3 Walking for 1 Teleporting,1,0,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3 Mounted for 1 Teleporting,1,0,0,0,0,-3,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,3 Initiator for 1 Melee,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,0,0,1
302,3 Pusher for 1 Melee,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,0,1
303,3 Jumping for 1 Melee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,0,1
304,3 Slithering for 1 Melee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-3,0,1


In [5]:
df_with_trading = pd.concat([df, trading_df], ignore_index=True)

In [6]:
required_tokens = {
    "Flying": 3,
    "Walking": 3,
    "Teleporting": 1,
    "Melee": 5,
}

best_solution = integer_linear_solver(df, required_tokens.items(), "DifficultyScore")

Solving with SAT
Number of variables = 124
Number of constraints = 4
Problem solved in 47.000000 milliseconds
Optimal objective value = 500.000000
Ember Spirit = 1.000000
Juggernaut = 2.000000
Nightstalker = 1.000000
Winter Wyvern = 1.000000
Advanced usage:
Problem solved in 0 branch-and-bound nodes




c:\projects\dota2-token-optimizer\helpers.py:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, row.to_frame().T])


In [7]:
best_solution_with_trading = integer_linear_solver(df_with_trading, required_tokens.items(), "DifficultyScore")

Solving with SAT
Number of variables = 430
Number of constraints = 4
The problem does not have an optimal solution!
5
